In [1]:
import pandas as pd 
import numpy as np 
import time 

import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras

from tensorflow import feature_column
from tensorflow.keras import layers

print(tf.__version__)


2.0.0-beta1


In [2]:
#samsung_df = pd.read_csv(r"galaxy_smallmatrix_labeled_9d.csv")
dataframe = pd.read_csv(r"iphone_smallmatrix_labeled_9d.csv")

In [3]:
dataframe.tail()

,iphone,samsunggalaxy,sonyxperia,nokialumina,htcphone,ios,googleandroid,iphonecampos,samsungcampos,sonycampos,...,sonyperunc,nokiaperunc,htcperunc,iosperpos,googleperpos,iosperneg,googleperneg,iosperunc,googleperunc,iphonesentiment
12968,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
12969,2,1,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12970,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
12971,2,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,5
12972,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5


In [4]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(dataframe, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')


8302 train examples
2076 validation examples
2595 test examples


In [5]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe,label, shuffle=True, batch_size=32):
    dataframe = dataframe.copy()
    labels = dataframe.pop(label)
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    return ds


In [6]:
features = dataframe.drop('iphonesentiment',axis = 1)

features = features.columns


In [7]:
train_ds = df_to_dataset(train,'iphonesentiment')
val_ds = df_to_dataset(val, shuffle=False,label = 'iphonesentiment' )
test_ds = df_to_dataset(test, shuffle=False, label =  'iphonesentiment')


In [8]:
for feature_batch, label_batch in train_ds.take(1):
    print('Every feature:', list(feature_batch.keys()))
    print('A batch of : nokialumina ', feature_batch['nokialumina'])
    print('A batch of targets:', label_batch )


Every feature: ['iphone', 'samsunggalaxy', 'sonyxperia', 'nokialumina', 'htcphone', 'ios', 'googleandroid', 'iphonecampos', 'samsungcampos', 'sonycampos', 'nokiacampos', 'htccampos', 'iphonecamneg', 'samsungcamneg', 'sonycamneg', 'nokiacamneg', 'htccamneg', 'iphonecamunc', 'samsungcamunc', 'sonycamunc', 'nokiacamunc', 'htccamunc', 'iphonedispos', 'samsungdispos', 'sonydispos', 'nokiadispos', 'htcdispos', 'iphonedisneg', 'samsungdisneg', 'sonydisneg', 'nokiadisneg', 'htcdisneg', 'iphonedisunc', 'samsungdisunc', 'sonydisunc', 'nokiadisunc', 'htcdisunc', 'iphoneperpos', 'samsungperpos', 'sonyperpos', 'nokiaperpos', 'htcperpos', 'iphoneperneg', 'samsungperneg', 'sonyperneg', 'nokiaperneg', 'htcperneg', 'iphoneperunc', 'samsungperunc', 'sonyperunc', 'nokiaperunc', 'htcperunc', 'iosperpos', 'googleperpos', 'iosperneg', 'googleperneg', 'iosperunc', 'googleperunc']
A batch of : nokialumina  tf.Tensor([0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0], shape=(32,), dtype=int32)
A

In [9]:
def demo(feature_column):
    feature_layer = layers.DenseFeatures(feature_column)
    print(feature_layer(example_batch).numpy())


In [10]:
feature_columns = []

# numeric 

for header in features :
    feature_columns.append(feature_column.numeric_column(header))


     

In [11]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)


In [12]:
len(features)

58

In [18]:
model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(128, activation='relu'),
  layers.Dense(128, activation='relu'),
  layers.Dense(6, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'],
              run_eagerly=True)




model.fit(train_ds,
          validation_data=val_ds,
          epochs=10,)


Epoch 1/10
260/260 [==============================] - 13s 51ms/step - loss: 40.2937 - accuracy: 0.1342 - val_loss: 40.8072 - val_accuracy: 0.6696
Epoch 2/10
260/260 [==============================] - 12s 48ms/step - loss: 40.2510 - accuracy: 0.1959 - val_loss: 40.7159 - val_accuracy: 0.0087
Epoch 3/10
260/260 [==============================] - 12s 48ms/step - loss: 40.2522 - accuracy: 0.1955 - val_loss: 40.6169 - val_accuracy: 0.0583
Epoch 4/10
260/260 [==============================] - 13s 49ms/step - loss: 40.2522 - accuracy: 0.1029 - val_loss: 40.6447 - val_accuracy: 0.5925
Epoch 5/10
260/260 [==============================] - 13s 48ms/step - loss: 40.2029 - accuracy: 0.2744 - val_loss: 40.6006 - val_accuracy: 0.1397
Epoch 6/10
260/260 [==============================] - 12s 48ms/step - loss: 40.2024 - accuracy: 0.0924 - val_loss: 40.6128 - val_accuracy: 0.0419
Epoch 7/10
260/260 [==============================] - 12s 48ms/step - loss: 40.1924 - accuracy: 0.1079 - val_loss: 40.5706 -